In [ ]:
import scipy.io as sio
from dataclasses import dataclass
from typing import List, Tuple
import os
from dotenv import load_dotenv
load_dotenv()
import tidy3d as td
from tidy3d import web
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from natsort import natsorted
import matplotlib
font = {
        'weight' : 'bold',
        'size'   : 27}

matplotlib.rc('font', **font)
import h5py
import plotly.graph_objs as go
import plotly
import plotly.express as px
from plotly.subplots import make_subplots
from IPython.display import display, HTML
plotly.offline.init_notebook_mode()
display(HTML(
    '<script type="text/javascript" async src="https://cdnjs.cloudflare.com/ajax/libs/mathjax/2.7.1/MathJax.js?config=TeX-MML-AM_SVG"></script>'
))
import pandas as pd
import sys
import os

# Assuming /AutomationModule is in the root directory of your project
sys.path.append(os.path.abspath(r'H:\phd stuff\tidy3d'))

from AutomationModule import * 

import AutomationModule as AM

In [ ]:
tidy3dAPI = os.environ["API_TIDY3D_KEY"]

In [ ]:
a=3.4

In [ ]:
sizes =    np.array([1])*14.3
perm_values = [2.3716,4.0,4.84,6.25,7.5625,9.0,11.56,12.9,16.0]
host_perm = [None,2.0164]
L_values = [1]
names = [
      "ak4_1000_eps_2p3716_phih5_0p30_phiL_0p29_size_",
      "ak4_1000_eps_4p0_phih5_0p30_phiL_0p29_size_",
      "ak4_1000_eps_4p84_phih5_0p30_phiL_0p29_size_",
      "ak4_1000_eps_6p25_phih5_0p30_phiL_0p29_size_",
      "ak4_1000_eps_7p5625_phih5_0p30_phiL_0p29_size_",
      "ak4_1000_eps_9p0_phih5_0p30_phiL_0p29_size_",
      "ak4_1000_eps_11p56_phih5_0p30_phiL_0p29_size_",
      "ak4_1000_eps_12p96_phih5_0p30_phiL_0p29_size_",
      "ak4_1000_eps_16p00_phih5_0p30_phiL_0p29_size_"
]


values_transmission_right = []
values_transmission_left = []

values_transmission_right_host = []
values_transmission_left_host = []



for k, perm in enumerate(perm_values):
      
      path_direction = rf"H:\phd stuff\tidy3d\data\02_27_2025 Florescu LSU H5 Transmission eps_perm_{perm}\z_incidence"

      folder_path = f"{path_direction}"

      perm_value_right = []
      perm_value_left = []
      perm_value_right_host = []
      perm_value_left_host = []
      for i,size in enumerate(L_values):
       for host in host_perm:
            filename = rf"{names[k]}{size}{('_bg_'+str(host)) if host else ""}.txt"
           
            if os.path.exists(os.path.join(folder_path, filename)):
                  print(filename)
            else:
                 continue

            
            if os.path.isfile(os.path.join(folder_path, filename)):
                file=os.path.join(folder_path, filename)
                structure_1 = AM.loadFromFile(key = tidy3dAPI, file_path=file,get_ref=True)
                sim_data, sim_data0 = structure_1.sim_data, structure_1.sim_data0
                transmission0_left = sim_data0['flux2'].flux
                transmission_left = sim_data['flux2'].flux
                transmission0_right = sim_data0['flux1'].flux
                transmission_right = sim_data['flux1'].flux
                transmission_normalized_left = transmission_left.values / transmission0_left.values
                transmission_normalized_right = transmission_right.values / transmission0_right.values
                monitor_lambdas = (a/(td.C_0/np.array(np.array(sim_data.simulation.monitors)[0].freqs)))
                eval(rf"perm_value_right{'_host' if host else ''}.append(transmission_normalized_right)")
                eval(rf"perm_value_left{'_host' if host else ''}.append(transmission_normalized_left)")

               

      values_transmission_right.append(perm_value_right)
      values_transmission_left.append(perm_value_left)
      values_transmission_right_host.append(perm_value_right_host)
      values_transmission_left_host.append(perm_value_left_host)



      
      # raise(Exception)
               
    
        


     

               



In [ ]:
data = {
                        'transmission_right':values_transmission_right, #shape (perm, thickness, freqs)
                        'transmission_left':values_transmission_left,
                        'nu(a/lambda)':monitor_lambdas,
                        "raw_freqs(Hz)":(sim_data.simulation.monitors)[0].freqs,
                        "sizes":sizes/14.3,
                        "cell_size":14.3,
                        "a":a,
                        "eps":perm_values
                        
                        
                  }


data_host = {
                        'transmission_right':values_transmission_right_host,
                        'transmission_left':values_transmission_left_host,
                        'nu(a/lambda)':monitor_lambdas,
                        "raw_freqs(Hz)":(sim_data.simulation.monitors)[0].freqs,
                        "sizes":sizes/14.3,
                        "cell_size":14.3,
                        "a":a,
                        "eps":perm_values,
                        "eps_host":[2.0164]
                        
                        
                  }


create_hdf5_from_dict(data,rf"H:\phd stuff\tidy3d\output\LSU Data\transmission_perms.h5")
create_hdf5_from_dict(data_host,rf"H:\phd stuff\tidy3d\output\LSU Data\transmission_perms_host.h5")
      